In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

In [2]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


#Les étapes pour créer une combinaison des modèles mentionnés ci-dessus sont les suivantes:
1. importer les quatre modèles et le jeu de données + prétraiter le jeu de données pour correspondre aux modèles.
2. choisissez au hasard une ligne à partir de celui-ci (au lieu de saisir les données manuellement)
3. Supprimez la colonne Reached On Time et extrayez le prix du produit de la ligne.
4. classer le produit dans l'un des modèles selon son prix.
5. imprimer si le produit est arrivé à temps ou non.

#1.  Importer les quatre modèles et le jeu de données + prétraiter le jeu de données pour qu'il corresponde aux modèles.

In [3]:
CA_model1=pickle.load(open('/content/Drive/MyDrive/context awareness models/Shipment tracking models/RF/model 1 optimized acc_74_449.sav', 'rb'))
CA_model2=pickle.load(open('/content/Drive/MyDrive/context awareness models/Shipment tracking models/RF/model 2 optimized acc_100_000.sav', 'rb'))
G_model=pickle.load(open('/content/Drive/MyDrive/e-comerce model/model_RF_ecommerce_73-875.sav', 'rb'))



In [4]:
df = pd.read_csv("/content/Drive/MyDrive/context awareness datasets/shipment tracking dataset/data.csv")


In [5]:
df.drop(['ID'], axis=1, inplace=True)

##############################################################################################################

df_reached = df[df['Reached.on.Time_Y.N'] == 0]
df_not_reached = df[df['Reached.on.Time_Y.N'] == 1]

###############################################################################################################

df_reached_downsampled = resample(df_reached,replace=False,n_samples=4000,random_state=25) 
df_not_reached_downsampled = resample(df_not_reached,replace=False,n_samples=4000,random_state=25) 
df_downsampled = pd.concat([df_not_reached_downsampled,df_reached_downsampled])

###############################################################################################################

df_downsampled.Product_importance[df_downsampled.Product_importance == 'low'] = 1
df_downsampled.Product_importance[df_downsampled.Product_importance == 'medium'] = 2
df_downsampled.Product_importance[df_downsampled.Product_importance == 'high'] = 3

df_downsampled.Gender[df_downsampled.Gender == 'F'] = 0
df_downsampled.Gender[df_downsampled.Gender == 'M'] = 1

###############################################################################################################

df_encoded=pd.get_dummies(df_downsampled,columns=['Warehouse_block','Mode_of_Shipment'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarn

In [6]:
df_encoded.head()

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship
2554,5,5,264,3,2,0,23,2837,1,0,0,1,0,0,0,0,1
2529,2,1,255,3,1,1,19,1551,1,0,1,0,0,0,0,0,1
1073,3,1,148,2,2,1,34,1053,1,0,0,0,0,1,0,1,0
9432,6,3,271,4,1,0,2,1364,1,0,0,0,1,0,0,1,0
2539,2,5,252,2,1,0,21,2070,1,0,0,0,0,1,0,0,1


#2. choisissez au hasard une ligne à partir de celui-ci (au lieu de saisir les données manuellement)  

In [7]:
data_point = df_encoded.sample()
data_point

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship
288,5,5,156,2,1,1,44,1561,1,0,0,0,1,0,1,0,0


#3. supprimez la colonne Reached On Time et extrayez le prix du produit de la ligne.

In [12]:
X=data_point.drop("Reached.on.Time_Y.N", axis = 1)
poids=X.iloc[0]['Weight_in_gms']
X

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship
288,5,5,156,2,1,1,44,1561,0,0,0,1,0,1,0,0


In [13]:
poids

1561

#4. classez le produit dans l'un des modèles en fonction de son prix.

In [16]:
if (poids <=2000 ):
  prediction=CA_model1.predict(X)
elif ((poids >2000 ) & (poids <=4000)):
  prediction=CA_model2.predict(X)
elif (poids >4000 ):
  prediction=G_model.predict(X)
 


#5. imprimer si le produit est arrivé à temps ou non.

In [17]:
if (prediction[0]==1) : 
  print("Le produit est arrivé a l'heure ✘")
else :
  print ("Le produit n'est pas arrivé a l'heure ✔ ")

Le produit est arrivé a l'heure ✘
